In [ ]:
# Libraries Required
import warnings
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
def data_ingestion():
    
    # Load data (already split into train/test)
    # X-train: (60000, 28, 28), y-train: (60000,) - 60000 samples of 28x28px images and their labels
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    
    # CNNs expects 4D input: (samples, height, width, channels)
    # Expand channel dimension to match Keras CNN expectation - grayscale images have 1 channel
    # (60000, 28, 28) -> (60000, 28, 28, 1)
    X_train = np.expand_dims(X_train, axis=-1).astype('float32')
    X_test  = np.expand_dims(X_test, axis=-1).astype('float32')
    
    # Normalize (Optional)
    X_train /= 255.0
    X_test  /= 255.0

    input_shape = (28, 28, 1) # Height, Width, Channels

    # One-hot encode labels - convert class vectors (0-9) to binary class vectors
    num_classes = len(np.unique(y_train))                               # 10 (digits 0–9)
    y_train = to_categorical(y_train, num_classes).astype('float32')    # Ensure dtype is float32
    y_test  = to_categorical(y_test, num_classes).astype('float32')  
    
    return X_train, y_train, X_test, y_test, input_shape, num_classes

X_train, y_train, X_test, y_test, input_shape, num_classes = data_ingestion()

In [ ]:
def create_model(input_shape, num_classes, filters1=32, filters2=64, dense_units=120, lr=0.001, optimizer_type='adam'):
    
    K.clear_session()  # erase previous model from memory
    # Builds a LeNet-style CNN (ReLU + MaxPooling)
    
    model = Sequential()

    # Layer 1: Conv -> ReLU -> MaxPool
    # Conv2D(filters1, 5x5) - Learn basic patterns
    # MaxPooling2D - Reduce spatial size, keep strongest features
    model.add(Conv2D(filters=filters1, kernel_size=(5,5), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Layer 2: Conv -> ReLU -> MaxPool
    # Conv2D(filters2, 5x5) - Learn more complex patterns
    # MaxPooling2D - Further reduce spatial size
    model.add(Conv2D(filters=filters2, kernel_size=(5,5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Flatten before Dense layers - convert 2D feature maps to 1D feature vectors
    model.add(Flatten())

    # Classification layers
    model.add(Dense(dense_units, activation='relu'))    # Dense(dense_units, ReLU) - Learn combinations of features
    model.add(Dense(num_classes, activation='softmax')) # Output class probabilities (digits 0–9)

    # Compile model with specified learning rate
    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [16]:
def cross_validate_model(X, y, input_shape, num_classes, filters1=32, filters2=64, dense_units=120, lr=0.001, epochs=5, batch_size=128):
    
    # Performs 5-fold stratified cross-validation on the training set using create_model().
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # y needs to be integers for StratifiedKFold
    y_integers = np.argmax(y, axis=1)
    fold_accuracies = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y_integers), 1):
        
        print(f"\n##### Fold {fold} #####")
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]
        
        # Build a fresh model for each fold
        model = create_model(input_shape, num_classes, filters1=filters1, filters2=filters2, dense_units=dense_units, lr=lr)
        
        # Early stopping to avoid overfitting
        early_stop = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
        
        # Train
        model.fit(
            X_train_fold, y_train_fold,
            validation_data=(X_val_fold, y_val_fold),
            epochs=epochs,
            batch_size=batch_size,
            verbose=1,
            callbacks=[early_stop]
        )
        
        # Evaluate on validation fold
        val_loss, val_acc = model.evaluate(X_val_fold, y_val_fold, verbose=0)
        print(f"Validation Accuracy: {val_acc:.4f}")
        fold_accuracies.append(val_acc)
    
    avg_accuracy = np.mean(fold_accuracies)
    print(f"\nAverage Validation Accuracy across 5 folds: {avg_accuracy:.4f}")
    
    return fold_accuracies, avg_accuracy

In [ ]:
### Hyperparameter Search ###

def hyperparameter_search(X_train, y_train, input_shape, num_classes):
    filter_sets = [(16, 32), (32, 64), (64, 128)]
    learning_rates = [0.001, 0.0005]

    results = []

    for filters1, filters2 in filter_sets:
        for lr in learning_rates:

            print(f"\n##### Testing filters: ({filters1}, {filters2}), lr: {lr} #####")
            fold_accuracies, avg_accuracy = cross_validate_model(
                X_train, y_train, input_shape, num_classes,
                filters1=filters1,
                filters2=filters2,
                dense_units=120,
                lr=lr,
                epochs=5,
                batch_size=128
            )
            results.append({
                'filters': (filters1, filters2),
                'learning_rate': lr,
                'fold_accuracies': fold_accuracies,
                'avg_accuracy': avg_accuracy
                })

    return results

warnings.filterwarnings('ignore')
results = hyperparameter_search(X_train, y_train, input_shape, num_classes)
best_accuracy = 0
best_result = None

for result in results:
    if result['avg_accuracy'] > best_accuracy:
        best_accuracy = result['avg_accuracy']
        best_result = result

print("\nBest hyperparameters found:")
print(f"Filters: {best_result['filters']}")
print(f"Learning rate: {best_result['learning_rate']}")
print(f"Average validation accuracy: {best_result['avg_accuracy']:.4f}")

In [ ]:
### Building Best Model ###

def train_final_model(X_train, y_train, X_test, y_test, input_shape, num_classes, best_result):

    # Extract best hyperparameters
    best_filters = best_result['filters']
    best_lr = best_result['learning_rate']

    print(f"\nTraining final model with hyperparameters:")
    print(f"Filters: {best_filters}, Learning Rate: {best_lr}")

    # Create and compile model with best hyperparameters
    model = create_model(
        input_shape=input_shape,
        num_classes=num_classes,
        filters1=best_filters[0],
        filters2=best_filters[1],
        dense_units=120,
        lr=best_lr
    )

    # Train on the full training data (60,000 images)
    history = model.fit(
        X_train, y_train,
        epochs=10,              
        batch_size=128,
        validation_split=0.1,   # small part (10%) used for internal validation
        verbose=1
    )

    # Evaluate on the test data
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nFinal Test Accuracy: {test_accuracy:.4f}")

    return model, history, test_accuracy

best_model, history, final_test_accuracy = train_final_model(X_train, y_train, X_test, y_test, input_shape, num_classes, best_result)

#### My CNN Architecture

The convolutional neural network (CNN) used in this project is based on the classic LeNet architecture, updated with modern techniques to improve performance on the MNIST-Digits dataset. It takes 28x28 grayscale images as input, where each pixel represents a level of brightness. The first layer is a convolutional layer with 64 filters of size 5x5. Think of these filters as small windows that scan the image to pick up simple patterns like edges or corners. The layer uses a ReLU activation, which helps the network learn more complex relationships by adding non-linearity. Next is a 2x2 max-pooling layer, which shrinks the image representation by keeping only the strongest signals in each small patch. This makes the model faster and more robust to small shifts or distortions in the image.

The second convolutional layer has 128 filters of the same size, again followed by ReLU and 2x2 max-pooling. This layer captures more detailed patterns, like loops and intersections that appear in handwritten digits. After the convolutional layers, the output is flattened into a 1D vector so it can be fed into a dense layer with 120 neurons. Each neuron in this layer looks at combinations of features from the previous layers to make sense of the image. Finally, the output layer has 10 neurons with a softmax activation, which converts the network’s predictions into probabilities for each digit (0–9). The model is trained using the Adam optimizer with a learning rate of 0.0005 and categorical crossentropy loss, which measures how far the predicted probabilities are from the actual labels. Overall, this architecture achieves a high accuracy on MNIST while being easy to understand and computationally efficient.

#### Hyperparameter Search

To find the best configuration for our CNN, a hyperparameter search was performed using 5-fold stratified cross-validation on the training set. Hyperparameters are settings that control how the network learns, such as the number of filters in each convolutional layer, which determine how many patterns the network can detect at each stage, and the learning rate, which controls how quickly the model updates its weights during training. In this project, three combinations of filters for the two convolutional layers were tested: 16 and 32, 32 and 64, and 64 and 128. Two learning rates, 0.001 and 0.0005, were explored while keeping the dense layer fixed at 120 neurons.

For each combination, the training data was split into five folds in a stratified manner, meaning each fold maintained roughly the same proportion of each digit class. The model was trained on four folds and validated on the remaining fold. This process was repeated so that each fold was used as a validation set once. The performance of the model was measured by the average validation accuracy across the five folds. This approach ensures that the chosen hyperparameters are not specific to a particular subset of the training data and helps prevent overfitting, which happens when a model learns the training data too well but performs poorly on new data.

The combination that achieved the highest average accuracy, specifically 64 and 128 filters with a learning rate of 0.0005, was chosen as the final configuration. Using cross-validation in this way helps guarantee that the selected hyperparameters generalize well to unseen data, providing a robust starting point for training the CNN on the full dataset.


#### Final Model Training & Test Accuracy

After finding the best hyperparameters, specifically 64 filters in the first convolutional layer, 128 filters in the second, and a learning rate of 0.0005, the CNN was retrained on the full MNIST training dataset, which contains 60,000 images of handwritten digits. During this final training, the model went through 10 epochs, meaning it saw the entire training dataset 10 times, and it processed the images in small groups called batches of 128 images at a time to make learning more efficient. A small portion of the training data, 10%, was set aside as a validation set to check how well the model was learning and to ensure it was not just memorizing the training images. After training, the model was tested on the separate set of 10,000 unseen images, achieving a final test accuracy of 99.14%, which measures the proportion of digits the model classified correctly. This high accuracy shows that the chosen architecture and hyperparameters allow the network to generalize well, meaning it can correctly recognize new handwritten digits it has never seen before.

#### Comparison with Current MNIST Classification Results

To contextualize this performance, it's valuable to compare it with some recent studies on MNIST digit classification:

- Hyperparameter Tuning (Kundu et al., 2025): This research systematically explored the effect of hyperparameter tuning on CNN performance. By adjusting the learning rate, batch size, and number of convolutional layers, and using the Adam optimizer, their model achieved a recognition rate of 99.89%. The study highlighted that adding extra convolutional layers improved performance by allowing the network to extract deeper and more complex features. This is very similar to what we have tried to achieve in this lab.
- Ensemble Methods: A study employing an ensemble of three simple CNN models with varying kernel sizes achieved a test accuracy of up to 99.87% on the MNIST dataset. This approach utilized data augmentation techniques, such as rotation and translation, to enhance model robustness.

#### References

1. "Introduction to Deep Learning." MIT Introduction to Deep Learning, Massachusetts Institute of Technology, 2024, https://introtodeeplearning.com/
2. Kundu, Roumo, Anurag Sinha, Biresh Kumar, Rohan Gautam, Mohammad Shahid Raza, and Syed Abid Hussain. "Exploration of Hyperparameter Tuning in Handwritten Digit Recognition Datasets Using CNN." F1000Research 14 (2025): 274. https://doi.org/10.12688/f1000research.161053.1
3. An, S., Lee, M., Park, S., Yang, H., & So, J. (2020). An ensemble of simple convolutional neural network models for MNIST digit recognition (Version 2). arXiv. https://doi.org/10.48550/arXiv.2008.10400